# GPT Practice

In [ ]:
# Check if torch can use cuda.
import torch

s1  = torch.cuda.is_available()
s2 = torch.cuda.device_count()
s3 = torch.cuda.current_device()
s4 = torch.cuda.get_device_name(0)
print(f'{s1}, {s2}, {s3}, {s4}')

In [ ]:
# Get dataset
import wget
from pathlib import Path

# Checks if dataset was already downloaded. Skips the download routine if dataset was detected.
dataset_path = Path.cwd() / 'input.txt'
if not dataset_path.exists():
    url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    filename = wget.download(url)
    print(f'Done downloading dataset {filename} from {url}')
else:
    print("Dataset already existed hence skipping download.")


In [ ]:
# Read data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    print(f'Length of dataset in characters: {len(text)}')

In [ ]:
# Look at the first 2000 characters
print(text[:2000])

In [ ]:
# Get the unique characters in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'The unique characters are {chars} \nvocab_size: {vocab_size}')

In [ ]:
# Tokenization
# Many ways to do this, you can use Sentencepiece by goole or tiktoken. 
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Test yow !")) 
print(decode(encode("Test yow !")))

In [ ]:
# Encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(f'Data shape: {data.shape}, type: {data.dtype}')
print(data[:1000])

In [ ]:
# Separate data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, the rest will be validation
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

In [ ]:
# Prints the context given a certain target
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
    
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('_________')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()}, the target: {target}")